In [1]:
import pandas as pd
import numpy as np
import requests
import json

Создадим функцию, которая будет определять сигнатуру по анализу DE.

In [2]:
def make_signature_from_DE (file, logFC = (-1, 1), pvalue = 0.01):
    """
    find genes with increased and decreased expression
    
    Parametrs
    ---------
    file: the path to the file with differential expression results (DESeq2, EdgeR)
    logFC: a tuple with a lower and upper threshold for log2 fold change
    pvalue: threshold for pvalue
    
    Return
    ------
    a tuple with 2 lists: 1) list of genes with increased expression - genes with a logarithm of the fold change greater than 
                            set upper threshold for logFC, pvalue less than threshold for pvalue
                          2) list of genes with reduced  expression - genes with a logarithm of the fold change less than 
                            set lower threshold for logFC, pvalue less than threshold for pvalue
    Each list of genes is sorted decrease of the modulo the logarithm of the fold change and sorted by increasing pvalue.
    """
    Dif_exp = pd.read_table(file, sep = '\t')
    
    if ('logFC' in Dif_exp.columns.tolist()) & ('PValue' in Dif_exp.columns.tolist()):# названия столбцов 'logFC', PValue' характерны для edgeR
        Dif_exp_up = Dif_exp[(Dif_exp['logFC'] > logFC[1]) & (Dif_exp['PValue'] < pvalue)]
        Dif_exp_up = Dif_exp_up.sort_values(by = ['logFC', 'PValue'], ascending= [False, True])
        Dif_exp_down = Dif_exp[(Dif_exp['logFC'] < logFC[0]) & (Dif_exp['PValue'] < pvalue)]
        Dif_exp_down = Dif_exp_down.sort_values(by = ['logFC', 'PValue'], ascending = [True, True])
    elif ('log2FoldChange' in Dif_exp.columns.tolist()) & ('pvalue' in Dif_exp.columns.tolist()):# названия столбцов 'log2FoldChange', 'pvalue' характерны для DESeq2
        Dif_exp_up = Dif_exp[(Dif_exp['log2FoldChange'] > logFC[1]) & (Dif_exp['pvalue'] < pvalue)]
        Dif_exp_up = Dif_exp_up.sort_values(by = ['log2FoldChange', 'pvalue'], ascending = [False, True])
        Dif_exp_down = Dif_exp[(Dif_exp['log2FoldChange'] < logFC[0]) & (Dif_exp['pvalue'] < pvalue)]
        Dif_exp_down = Dif_exp_down.sort_values(by = ['log2FoldChange', 'pvalue'], ascending = [True, True])
        
    Dif_exp_up_genes = Dif_exp_up.index.tolist()
    Dif_exp_down_genes = Dif_exp_down.index.tolist()
    return (Dif_exp_up_genes, Dif_exp_down_genes, )

Для примера посмотрим на результаты анализа дифференциальной экспрессии для образцов клеток сердца и фибробластов c помощью edgeR.

Посмотрим сам файл с результатами.

In [4]:
data = pd.read_table('DATA/DE_heart_fibroblast/DE_with_edgeR_for_heart_fibroblast.txt', sep = '\t')
data.head()

,logFC,logCPM,PValue
A1BG,-1.616721,3.216281,3.648526e-26
A1CF,0.733942,-1.202507,2.740174e-04
A2M,3.911215,8.547616,7.819607e-114
A2ML1,-1.043900,0.707597,2.489561e-12
A2MP1,6.370985,1.889518,2.858788e-226


Определим гены с повышенной и пониженной экспрессией.

In [5]:
data_up_genes, data_down_genes = make_signature_from_DE('DATA/DE_heart_fibroblast/DE_with_edgeR_for_heart_fibroblast.txt')
print(len(data_up_genes), data_up_genes)

7492 ['MYH7', 'MYL2', 'MYH6', 'MB', 'MYBPC3', 'IGKC', 'NRAP', 'C14ORF180', 'CCNL1', 'IGLC1', 'IGLC3', 'NPPA', 'CSRP3', 'HHATL', 'C1QB', 'IGLC2', 'MYL3', 'IGHA2', 'C1QC', 'ITLN1', 'VSIG4', 'C1QA', 'IGHG2', 'CCL14', 'LMOD2', 'PPP1R3A', 'BMP10', 'COX6A2', 'TCAP', 'IGHG4', 'IGHG3', 'CASQ2', 'S100A8', 'IGHG1', 'GIMAP7', 'PLN', 'DHRS7C', 'S100A9', 'XIRP2', 'CALML3', 'FCGR2A', 'CLIC5', 'NKX2-5', 'IGHM', 'SMPX', 'GIMAP4', 'GPIHBP1', 'MYL7', 'CKM', 'TNNI3', 'TYROBP', 'CTD-2587H24.4', 'GIMAP6', 'ASB10', 'SPARCL1', 'IGHA1', 'RD3L', 'PLA2G2A', 'PGAM2', 'REXO1L6P', 'PLVAP', 'LINC01330', 'CKMT2', 'LCN6', 'ADGRF5', 'FCGR3A', 'C19ORF48', 'MYOM2', 'TTN', 'C6', 'IGHGP', 'RP11-1221G12.2', 'GIMAP1-GIMAP5', 'TRIM63', 'MYO18B', 'GIMAP8', 'FOLR2', 'MYBPC1', 'SLC5A1', 'MYOM1', 'C10ORF71', 'TNNC1', 'SLN', 'HBB', 'HMGCS2', 'ACTN2', 'GIMAP5', 'RP11-96A1.4', 'MYOZ2', 'HCG22', 'XIRP1', 'CCL21', 'LMAN1L', 'TNNI3K', 'POPDC2', 'C11ORF21', 'CCL19', 'UGT2B4', 'TNNT2', 'NPY6R', 'MNDA', 'CCL18', 'SLC26A9', 'MYZAP', 'S100

In [6]:
print(len(data_down_genes), data_down_genes)

6770 ['MMP3', 'KRTAP2-3', 'KRTAP1-5', 'KRT34', 'TERT', 'AC004231.7', 'PSG4', 'LCE2A', 'SERPINB7', 'RP11-54D18.4', 'PSG5', 'RP11-73B8.2', 'OTX2', 'HOXA13', 'SLC7A3', 'NPTX1', 'ZIC2', 'CYP26A1', 'GREM1', 'HOXC8', 'FAM166A', 'LCE1F', 'C6ORF15', 'CER1', 'HOXC11', 'DKK1', 'CCDC74BP1', 'RPSAP75', 'EN1', 'ECEL1P1', 'CEMIP', 'HOXC9', 'KCNK2', 'EIF2S2P4', 'PLA2G3', 'SUMO2P6', 'SRPK2P', 'PRSS12', 'TMEM8C', 'CBX3P9', 'KRTAP1-1', 'HOXA11', 'NKX1-2', 'CAMKV', 'SLC6A15', 'PCSK9', 'HOXC10', 'FMN2', 'LRRC15', 'STMN2', 'ZIC3', 'AC104131.1', 'LIN28A', 'MAGEC2', 'EIF2S2P2', 'DAZL', 'UBL4B', 'FGF19', 'KRT33B', 'RP11-807H22.6', 'RPL23AP95', 'FOXQ1', 'SBSN', 'SERPINB2', 'PITX1', 'ZIC4', 'FOXL2', 'TRIM71', 'HES3', 'FOXA3', 'PSG1', 'GALNT5', 'KLRG2', 'HSPE1P2', 'LCE3D', 'PDCD5P1', 'RPSAP1', 'OR2A25', 'FOXF1', 'KPRP', 'MKX', 'NLRP10', 'L1TD1', 'EIF4A1P13', 'IL13RA2', 'LARGE2', 'FOXE1', 'HOXD13', 'VRTN', 'SEMG1', 'GLDC', 'GREM2', 'RPSAP52', 'SYT6', 'RP11-382N13.3', 'RAB3B', 'OXTR', 'SP9', 'WNT16', 'NKX2-3', 'MY

Сделаем API запрос.

In [6]:
url = 'https://maayanlab.cloud/L1000CDS2/query'
def upperGenes(genes):
    # The app uses uppercase gene symbols. So it is crucial to perform upperGenes() step.
    return [gene.upper() for gene in genes]

# gene-set search example
data = {"upGenes": data_up_genes,
        "dnGenes": data_down_genes}
data['upGenes'] = upperGenes(data['upGenes'])
data['dnGenes'] = upperGenes(data['dnGenes'])
config = {"aggravate": True, "searchMethod": "geneSet",
          "share": True, "combination": True,
          "db-version": "latest"}

#metadata = [{"key": "Tag", "value": "gene-set python example"}, {"key": "Cell", "value": "MCF7"}]
payload = {"data": data, "config": config}
headers = {'content-type': 'application/json'}
r = requests.post(url, data=json.dumps(payload), headers = headers)
resGeneSet = r.json()

Посмотрим perturbation name, PubChem ID, combinations.

In [7]:
try:
    for i in resGeneSet['topMeta']:
        if 'pert_desc' in i:
            print(i['pert_desc'])
except KeyError:
    print("В запросе нет данного ключа")

PP-110
PP-110
PLX-4032
AS605240
-666
BMS-536924
AS605240
BMS-536924
canertinib
-666
selumetinib
PD-184352
PLX-4720
canertinib
TG101348
TG101348
trametinib
PLX-4032
AZD-8330
PD-0325901
-666
palbociclib
-666
PD-0325901
selumetinib
PHA-665752
selumetinib
BMS-754807
trametinib
BMS-536924
-666
PHA-793887
TGX-221
trametinib
trametinib
PD-0325901
palbociclib
PD-184352
foretinib
PD-0325901
PD-0325901
vorinostat
erlotinib
gefitinib
QUINACRINE HYDROCHLORIDE
AZD-8330
PD-0325901
palbociclib
selumetinib
Nutlin-3


In [8]:
try:
    for i in resGeneSet['topMeta']:
        if 'pert_id' in i:
            print(i['pert_id'])
except KeyError:
    print("В запросе нет данного ключа")

BRD-K03618428
BRD-K03618428
BRD-K56343971
BRD-K41895714
BRD-K57080016
BRD-K34581968
BRD-K41895714
BRD-K34581968
BRD-K50168500
BRD-K53414658
BRD-K57080016
BRD-K05104363
BRD-K16478699
BRD-K50168500
BRD-K12502280
BRD-K12502280
BRD-K12343256
BRD-K56343971
BRD-K37687095
BRD-K49865102
BRD-K68548958
BRD-K51313569
BRD-K57080016
BRD-K49865102
BRD-K57080016
BRD-K95435023
BRD-K57080016
BRD-K13049116
BRD-K12343256
BRD-K34581968
BRD-K98490050
BRD-K64800655
BRD-A41692738
BRD-K12343256
BRD-K12343256
BRD-K49865102
BRD-K51313569
BRD-K05104363
BRD-K03449891
BRD-K49865102
BRD-K49865102
BRD-K81418486
BRD-K70401845
BRD-K64052750
BRD-A45889380
BRD-K37687095
BRD-K49865102
BRD-K51313569
BRD-K57080016
BRD-A12230535


In [9]:
try:
    for i in resGeneSet['topMeta']:
        if 'pubchem_id' in i:
            print(i['pubchem_id'])
except KeyError:
    print("В запросе нет данного ключа")

24905203
24905203
42611257
10377751
10127622
11353973
10377751
11353973
156414
9911830
10127622
6918454
24180719
156414
16722836
16722836
11707110
42611257
16666708
9826528
1285940
5330286
10127622
9826528
10127622
10461815
10127622
24785538
11707110
11353973
3926765
46191454
9907093
11707110
11707110
9826528
5330286
6918454
42642645
9826528
9826528
5311
176871
123631
23581813
16666708
9826528
5330286
10127622
216345


In [10]:
try:
    for i in resGeneSet['combinations']:
        print(i)
except KeyError:
    print("В запросе нет данного ключа")

{'X1': 'CPC006_HT29_24H:BRD-K03618428:22.2', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0648}
{'X1': 'CPC006_A375_24H:BRD-K03618428:22.2', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0615}
{'X1': 'CPC006_A375_24H:BRD-K56343971:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0609}
{'X1': 'CPC006_A375_24H:BRD-K57080016:80.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0606}
{'X1': 'CPC006_A375_24H:BRD-K41895714:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0604}
{'X1': 'CPC006_A375_24H:BRD-K34581968:11.1', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.06}
{'X1': 'CPC006_HT29_24H:BRD-K41895714:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0598}
{'X1': 'CPC006_HT29_24H:BRD-K34581968:11.1', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0585}
{'X1': 'LJP005_HT29_24H:BRD-K05104363:10', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0574}
{'X1': 'CPC006_A375_24H:BRD-K53414658:10.0', 'X2':